<a href="https://colab.research.google.com/github/RobertoCuellar2019/tesis/blob/codigos_reconstruccion/v1_reconstruccion_no_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VERSIÓN 1.0 CÓDIGO TESIS DE PREGRADO**

RECONSTRUCCIÓN DE OBJETOS 3D UTILIZANDO HOLOGRAMAS FRACCIONARES Y REDES NEURONALES PARA EL RECONOCIMIENTO DE PATRONES DE DIFRACCIÓN

ROBERTO ANTONIO CUELLAR LOZANO
UNIVERSIDAD DE PAMPLONA 
PAMPLONA- NORTE DE SANTANDER 


In [0]:
import numpy as np  # libreria de calculo numerico y arreglos
import matplotlib.pyplot as pl # libreria para visualización

In [0]:
# Parámetros del modulador y el tamaño del pixel de modulador
M = 768 # numero de pixeles del modulador
N = 768
pi = np.pi
dx = 19e-6 # tamano del modulador
LX = M*dx
LY = M*dx
x = np.linspace(-LX/2,(LX/2)-dx, M) # dominio espacial modulador
y = x
l_onda = 514e-9 # longitud de onda
k = 2*pi/l_onda # numero de onda